# In The Beginning: A Social Media Logic Puzzle

My friend posted this simple puzzle online:

<p style="text-align: center;">
<img src="resources/original_puzzle.jpg" alt="Example Image" width="500">
</p>

I wanted to get better at rapidly solving logic puzzles, so I wrote a solver in **Python3**.


<div style="height:100px;"></div>

<div style="height:200px;"></div>

## My initial strategy was:

<div style="height:100px;"></div>

**Don't** try to write rules for how to solve the problem.

Instead:
* **Describe** what a bad solution looks like
* Try all the solutions.
  
<div style="height:300px;"></div>

In [ ]:
# STEP ONE: Tell if a particular 3-digit number breaks any of our rules

def check_guess(number):
    # ============= Break the number (an int 0-999) into its 3 component digits ============
    a,b,c = map(int, f"{number:03}")
    
    # State the problem
    disallowed = [6,9,5]
    one_correct_but_misplaced = [
        [8,9,1], 
        [0,1,4], 
        [3,7,2]
    ] 
    one_correct = [5,2,8]

    # ==================== Check all rules, Fail as soon as 1 breaks ========================
    # Is there a disallowed digit?
    if a in disallowed or b in disallowed or c in disallowed:
        return False

    # Does our guess have one of the correct but misplaced digits in a different position?
    for p in one_correct_but_misplaced:
        # We were in the same place, so this guess is bad.
        if a == p[0] or b == p[1] or c == p[2]:
            return False
    num_a = p.count(a)
    num_b = p.count(b)
    num_c = p.count(c)
    if num_a + num_b + num_c != 1:
        # We wanted only 1 digit to be off. This guess is bad.
        return False

    # Check and see if we had one of the correct digits
    a_match = a == one_correct[0]
    b_match = b == one_correct[1]
    c_match = c == one_correct[2]
    if sum([a_match, b_match, c_match]) == 1:
        # We had exactly one match! This was the last rule we had to match, so this is a valid solution!
        return True

    # Not exactly one match, so this guess was a dud.
    return False


In [ ]:
check_guess(100)

In [ ]:
# STEP TWO: Try every number 0-999

print("The following are possible solutions to the lock problem:\n")
for i in range(1000):
    if check_guess(i):
        print(f"\t{i}")


<div style="height:100px;"></div>

## Problem Space Size

<div style="height:100px;"></div>

We were able to do this via brute force because:
* the **domain** of each **variable** is small (10)
* there are only 3 variables

## In general
$$\text{Problem Space} = \prod_{i=1}^n D_i$$

Where:
$$D_i \text{ represents the domain of the }i^{th}\text{ variable.}$$
$$ n \text{ is the total number of variables.}$$

## So in this problem

$$\text{Problem Space} = \prod_{i=1}^3 10 = 1000$$


<div style="height:200px;"></div>

## ...But what if you aren't dealing with Numbers?

<div style="height:150px;"></div>

<p style="text-align: center;">
<img src="resources/solitaire_peg_puzzle.jpg" alt="Example Image" width="300">
</p>

<div style="height:150px;"></div>

## Our updated paradigm

<div style="height:150px;"></div>

**Don't** try to write rules for how to solve the problem.

**Instead:**

* Describe the starting state of the problem
* Decribe the rules for how to change state
* Describe what the goal (success) looks like
* **LET THE SOLVER EXPLORE ALL STATES (until it finds a success state)**
* (optional) Have the solver report back the path it took to get there
  
## Examples: Depth-First or Breadth-First Search

<p style="text-align: center;">
<img src="resources/depth_first_breadth_first.gif" alt="Example Image" width="500">
</p>

<div style="height:150px;"></div>

## It turns out there is a very powerful (and concise) framework for doing exactly this.

<div style="height:150px;"></div>

[![Modern Problem Solvers (Raymond Hettinger)](https://img.youtube.com/vi/_GP9OpZPUYc/0.jpg)](https://www.youtube.com/watch?v=_GP9OpZPUYc)

## Raymond Hettinger's 2019 Talk (Modern Problem Solvers)

[Slide deck and Code](https://rhettinger.github.io/overview.html)

<div style="height:150px;"></div>

In [ ]:
''' Generic Puzzle Solving Framework

License:  MIT
Author:   Raymond Hettinger

Simple Instructions:
====================

Create your puzzle as a subclass of Puzzle().
The first step is to choose a representation of the problem
state preferably stored as a string.  Set 'pos' to the starting
position and 'goal' to the ending position.  Create an __iter__()
method that computes all possible new puzzle states reachable from
the current state.  Call the .solve() method to solve the puzzle.

Important Note:

The __iter__() method must return a list or generator of puzzle
instances, not their representations.

Advanced Instructions:

1. .solve(depthFirst=1) will override the default breadth first search.
Use depth first when the puzzle known to be solved in a fixed number
of moves (for example, the eight queens problem is solved only when
the eighth queen is placed on the board; also, the triangle tee problem
removes one tee on each move until all tees are removed).  Breadth first
is ideal when the shortest path solution needs to be found or when
some paths have a potential to wander around infinitely (i.e. you can
randomly twist a Rubik's cube all day and never come near a solution).

2. Define __repr__ for a pretty printed version of the current position.
The state for the Tee puzzle looks best when the full triangle is drawn.

3. If the goal state can't be defined as a string, override the isgoal()
method.  For instance, the block puzzle is solved whenever block 1 is
in the lower left, it doesn't matter where the other pieces are; hence,
isgoal() is defined to check the lower left corner and return a boolean.

4. Some puzzle's can be simplified by treating symmetric positions as
equal.  Override the .canonical() method to pick one of the equilavent
positions as a representative.  This allows the solver to recognize paths
similar ones aleady explored.  In tic-tac-toe an upper left corner on
the first move is symmetrically equivalent to a move on the upper right;
hence there are only three possible first moves (a corner, a midde side,
or in the center).
'''

from collections import deque
from sys import intern
import re

class Puzzle:

    pos = ""                    # default starting position

    goal = ""                   # ending position used by isgoal()

    def __init__(self, pos=None):
        if pos: self.pos = pos

    def __repr__(self):         # returns a string representation of the position for printing the object
        return repr(self.pos)

    def canonical(self):        # returns a string representation after adjusting for symmetry
        return repr(self)

    def isgoal(self):
        return self.pos == self.goal

    def __iter__(self):         # returns list of objects of this class
        if 0: yield self

    def solve(pos, depthFirst=False):
        queue = deque([pos])
        trail = {intern(pos.canonical()): None}
        solution = deque()
        load = queue.append if depthFirst else queue.appendleft

        while not pos.isgoal():
            for m in pos:
                c = m.canonical()
                if c in trail:
                    continue
                trail[intern(c)] = pos
                load(m)
            pos = queue.pop()

        while pos:
            solution.appendleft(pos)
            pos = trail[pos.canonical()]

        return list(solution)

In [ ]:
class TriangleSolitaire(Puzzle):
    '''
    Given a Cracker Barrel puzzle board, figure out the sequence of moves
    That result in only a single peg being left in.
    '''

    # Initial triangle (1 = peg in, 0 = peg out)
    pos = [
        [1],
        [1,1],
        [1,1,1],
        [1,1,1,1],
        [1,1,1,0,1],
    ]

    # Test to see if we found a good state!
    def isgoal(self):
        return sum(row.count(1) for row in self.pos) == 1


    # Print the triangle state nicely
    def __repr__(self):
        ans = "\n"
        for r in self.pos:
            ans = ans + str(r) + "\n"
        return ans

    # Generator that yeilds all legal moves from the current position
    def __iter__(self):
        # Define possible moves as (delta_row, delta_col, jump_row, jump_col).
        # These are relative offsets from the starting position (r, c).
        # For example, (-2, -2, -1, -1) means:
        # Move up-left: final position is 2 rows up and 2 columns left,
        # and we jump over a peg one row up and one column left.
        directions = [
            (-2, -2, -1, -1), # up-left
            (-2,  0, -1,  0), # up-right
            ( 0, -2,  0, -1), # left
            ( 0,  2,  0,  1), # right
            ( 2,  0,  1,  0), # down-left
            ( 2,  2,  1,  1)  # down-right
        ]

        num_rows = len(self.pos)

        for r in range(num_rows):
            for c in range(len(self.pos[r])):
                # Check if there's a peg at the start position
                if self.pos[r][c] == 1:
                    for (dr, dc, jr, jc) in directions:
                        new_r = r + dr
                        new_c = c + dc
                        jump_r = r + jr
                        jump_c = c + jc
                        # Check board boundaries
                        if (0 <= new_r < num_rows and
                            0 <= jump_r < num_rows and
                            0 <= new_c < len(self.pos[new_r]) and
                            0 <= jump_c < len(self.pos[jump_r])):
    
                            # Check if we can make a valid jump
                            # Start: peg(1), Jump-over: peg(1), Destination: empty(0)
                            if self.pos[jump_r][jump_c] == 1 and self.pos[new_r][new_c] == 0:
                                # Create a new board configuration from the current one
                                new_pos = [row[:] for row in self.pos]

                                # Apply the move
                                new_pos[r][c] = 0       # vacate original spot
                                new_pos[jump_r][jump_c] = 0 # remove jumped peg
                                new_pos[new_r][new_c] = 1  # place peg in new spot

                                yield TriangleSolitaire(new_pos)


if __name__ == '__main__':
    from pprint import pprint
    pprint(TriangleSolitaire().solve(depthFirst=True))

<div style="height:150px;"></div>

## How does this work against Sudoku?

<div style="height:150px;"></div>

Here is a Sudoku puzzle that I lifted from my daughter's book **"Brain Games: Sudoku"**

(Puzzle 51 in the book)

<p style="text-align: center;">
<img src="resources/sudoku_puzzle_01.jpg" alt="Example Image" width="300">
</p>


<div style="height:150px;"></div>



<div style="height:150px;"></div>

## ANSWER: SUCKY.

<div style="height:150px;"></div>

This is a **MUCH LARGER** problem space

Sudoku uses **81 squares**, each of which holds a single integer. Valid entries are **1-9**.

If I took a completely blank sudoku puzzle, and assigned random valid integers into all the squares
(ignoring all other Sudoku rules), the total number of possible combinations is:

## Sudoku Problem Space

$$\text{Problem Space} = \prod_{i=1}^{81} 9 = 9^{81} = 1.966 * 10^{77}$$

*That is a **big-ass number**. We are not going to have luck brute forcing such a puzzle.*

## We also don't have any good rules for how to get from one state to the next.


<div style="height:150px;"></div>


<div style="height:150px;"></div>

## Here is our new paradigm:


<div style="height:150px;"></div>

**Don't** try to write rules for how to solve the problem.

**Instead:**

* Describe the starting state of the problem
* Describe the **variables** and what legal values (**domain**) thet can have
* Describe the **constraints** (what is **not allowed**)
* **LET THE SOLVER FIGURE OUT HOW TO GET THERE**

## This is called a CONSTRAINT SATISFACTION PROBLEM (CSP) SOLVER


<div style="height:150px;"></div>


In [ ]:
# DESCRIBE THE STARTING STATE OF THE PROBLEM

# 0 means an unknown value (a variable), which our solver will have to determine.

initial_puzzle_state = [
    [0,0,5,3,0,7,8,0,0],
    [0,0,0,9,0,0,5,0,0],
    [2,7,3,8,0,0,1,0,6],
    [6,0,0,0,0,0,2,7,1],
    [0,0,0,0,9,0,0,0,0],
    [8,1,4,0,0,0,0,0,3],
    [5,0,2,0,0,3,6,1,8],
    [0,0,6,0,0,2,0,0,0],
    [0,0,8,1,0,9,3,0,0],
]

In [ ]:
# DESCRIBE THE 81 VARIABLES AND THEIR DOMAINS

import constraint

problem = constraint.Problem()

init_vals = {}
for row_count, row in enumerate(initial_puzzle_state, start=1):
    for col_count, val in enumerate(row, start=1):
        if val != 0:
            init_vals[(row_count, col_count)] = val

for row in range(1, 10):
    for col in range(1, 10):
        init_val = init_vals.get((row, col))
        if init_val:
            problem.addVariable((row, col), [init_val])
        else:
            problem.addVariable((row, col), range(1, 10))


In [ ]:
# DESCRIBE THE CONSTRAINTS

# ==================== Sudoku general constraints =================================================

# CONSTRAINT 1: All rows must have different integers in every variable
for row in range(1, 10):
    problem.addConstraint(constraint.AllDifferentConstraint(), [(row, col) for col in range(1, 10)])
    # TODO: Find out why this is so much slower:
    # problem.addConstraint(lambda *args: len(set(args)) == 9, [(row, col) for col in range(1, 10)])

# CONSTRAINT 2: All cols must have different integers in every variable
for col in range(1, 10):
    problem.addConstraint(constraint.AllDifferentConstraint(), [(row, col) for row in range(1, 10)])

# CONSTRAINT 3: All 3x3 matrices must have different integers in every variable
for matrix_row in range(1, 10, 3):
    for matrix_col in range(1, 10, 3):
        var_list = []
        for row in range(3):
            for col in range(3):
                var_list.append((matrix_row + row, matrix_col + col))
        problem.addConstraint(constraint.AllDifferentConstraint(), var_list)


In [ ]:
# LET THE SOLVER DO ITS THING

from datetime import datetime


# Find a solution based on all the variables, their domains, and the constraints!
start_time = datetime.now()
solution = problem.getSolution()
end_time = datetime.now()


# Print results (solutions are not in sorted order otherwise)
print(f"(Solution took {(end_time - start_time)} seconds)\n\n")
for row in range(1, 10):
    line = ""
    for col in range(1, 10):
        line += f"{solution[(row, col)]}"
    print(line)


<div style="height:150px;"></div>

## Next Step: SAT Solvers


<div style="height:150px;"></div>

How do we solve for constraints that are logic based?

And will it still work even if we have **millions of variables**?


<div style="height:150px;"></div>



<div style="height:150px;"></div>

## Buzzwords:

* **SAT** : Boolean (True | False) **SAT**isfiability

* **Satisfiability** : "Is there **at least one** combination of inputs that makes the statement **True** ?

* **SAT Problems** must be in a **very specific form**: **CNF**

* **CNF** : **C**onjunctive **N**ormal **F**orm.

* **Conjunctive** : Boolean **AND**

* **Disjunctive** : Boolean **OR**

* Top level **AND**s, multiple **OR**-only clauses


<div style="height:150px;"></div>

<div style="height:150px;"></div>

## Example of Restating a Problem in CNF form

<div style="height:150px;"></div>

**English**: *“If the tire is flat then I will have to remove it and take it to the gas station.”*

**CNF** Form: **ALL** of the following **Clauses** need to be **True**:

* "the tire is not flat, **OR** I need to remove it."
* "the tire is not flat, **OR** I need to take it to the gas station."

(¬TireIsFlat ∨ NeedToRemove) ∧ (¬TireIsFlat ∨ TakeToGasStation)

<div style="height:150px;"></div>

<div style="height:150px;"></div>

## Einstein Puzzle


Reportedly this puzzle is from Albert Einstein who is said to have remarked that fewer than 2% of the population can solve this puzzle (this is lore, neither fact is true).
Entities

    There are five houses in unique colors: Blue, green, red, white and yellow.
    In each house lives a person of unique nationality: British, Danish, German, Norwegian and Swedish.
    Each person drinks a unique beverage: Beer, coffee, milk, tea and water.
    Each person smokes a unique cigar brand: Blue Master, Dunhill, Pall Mall, Prince and blend.
    Each person keeps a unique pet: Cats, birds, dogs, fish and horses.

<div style="height:150px;"></div>

<div style="height:150px;"></div>

## Constraints

    The Brit lives in a red house.
    The Swede keeps dogs as pets.
    The Dane drinks tea.
    The green house is on the left of the white, next to it.
    The green house owner drinks coffee.
    The person who smokes Pall Mall rears birds.
    The owner of the yellow house smokes Dunhill.
    The man living in the house right in the center drinks milk.
    The Norwegian lives in the first house.
    The man who smokes blend lives next to the one who keeps cats.
    The man who keeps horses lives next to the man who smokes Dunhill.
    The owner who smokes Blue Master drinks beer.
    The German smokes Prince.
    The Norwegian lives next to the blue house.
    The man who smokes blend has a neighbor who drinks water.

<div style="height:150px;"></div>

<div style="height:150px;"></div>

## Goal

The question you need to answer is: “Who keeps fish?”

<div style="height:150px;"></div>


In [ ]:
from resources.sat_utils import solve_one, from_dnf, one_of
from sys import intern
from pprint import pprint

houses =     ['1',          '2',      '3',           '4',         '5'       ]

groups = [
             ['dane',      'brit',   'swede',       'norwegian', 'german'   ],
             ['yellow',    'red',    'white',       'green',     'blue'     ],
             ['horse',     'cat',    'bird',        'fish',      'dog'      ],
             ['water',     'tea',    'milk',        'coffee',    'root beer'],
             ['pall mall', 'prince', 'blue master', 'dunhill',   'blends'   ],
]

values = [value for group in groups for value in group]

def comb(value, house):
    'Format how a value is shown at a given house'
    return intern(f'{value} {house}')

def found_at(value, house):
    'Value known to be at a specific house'
    return [(comb(value, house),)]

def same_house(value1, value2):
    'The two values occur in the same house:  brit1 & red1 | brit2 & red2 ...'
    return from_dnf([(comb(value1, i), comb(value2, i)) for i in houses])

def consecutive(value1, value2):
    'The values are in consecutive houses:  green1 & white2 | green2 & white3 ...'
    return from_dnf([(comb(value1, i), comb(value2, j))
                     for i, j in zip(houses, houses[1:])])

def beside(value1, value2):
    'The values occur side-by-side: blends1 & cat2 | blends2 & cat1 ...'
    return from_dnf([(comb(value1, i), comb(value2, j))
                     for i, j in zip(houses, houses[1:])] +
                    [(comb(value2, i), comb(value1, j))
                     for i, j in zip(houses, houses[1:])]
                    )

cnf = []

# each house gets exactly one value from each attribute group
for house in houses:
    for group in groups:
        cnf += one_of(comb(value, house) for value in group)

# each value gets assigned to exactly one house
for value in values:
    cnf += one_of(comb(value, house) for house in houses)

cnf += same_house('brit', 'red')
cnf += same_house('swede', 'dog')
cnf += same_house('dane', 'tea')
cnf += consecutive('green', 'white')
cnf += same_house('green', 'coffee')
cnf += same_house('pall mall', 'bird')
cnf += same_house('yellow', 'dunhill')
cnf += found_at('milk', 3)
cnf += found_at('norwegian', 1)
cnf += beside('blends', 'cat')
cnf += beside('horse', 'dunhill')
cnf += same_house('blue master', 'root beer')
cnf += same_house('german', 'prince')
cnf += beside('norwegian', 'blue')
cnf += beside('blends', 'water')

pprint(solve_one(cnf))